In [1]:
%env TF_CPP_MIN_LOG_LEVEL=3

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from tqdm.notebook import tqdm
from pathlib import Path
import itertools

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, multilabel_confusion_matrix
from sklearn.utils import shuffle

import tensorflow as tf

from importlib import reload
import sentinel_utils
import keras_model_creator
import plot_utils

from data_generator import DataGenerator

pd.options.mode.copy_on_write = True

env: TF_CPP_MIN_LOG_LEVEL=3


In [2]:
reload(sentinel_utils)
utils = sentinel_utils.SentinelUtils(min_occurrences=20000)

selected_sentinel = ['B3', 'B6', 'B8', 'B11']

In [3]:
model_parent_dir = Path('models', 'soilgrid_elevation')

loss = 'binary_crossentropy'
batch_size = 64
base_filters = 32
shards_dir = Path.home().joinpath('sentinel_data', 'shards')

seasons = ['06']
selected_classes = utils.get_processed_labels()
data_summary = utils.get_data_summary(
    shards_dir, seasons, selected_classes)

fixed_params = dict(
    shards_dir=shards_dir,
    seasons=seasons,
    selected_classes=selected_classes,
    loss=loss,
    data_summary=data_summary,
    batch_size=batch_size,
    base_filters=base_filters,
    dropout=0.2,
    epochs=10,
    overwrite=False,
    verbose=1,
    print_log=False
)

In [ ]:
reload(keras_model_creator)

band_groups = [
    selected_sentinel,
    selected_sentinel + utils.soilgrids_bands,
    selected_sentinel + utils.elevation_band,
    selected_sentinel + utils.soilgrids_bands + utils.elevation_band, 
]
names = ['sentinel', 'soil', 'elevation', 'all']

for name, band_group in zip(names, band_groups):
    band_indices = [utils.all_bands.index(b) for b in band_group]
           
    model_dir = model_parent_dir.joinpath(
        f'{loss}-{len(selected_classes.index)}'
        f'-{selected_classes.shape[1]}-{len(band_indices)}'
        f'-{"_".join(seasons)}-{batch_size}-{base_filters}-{name}'
    )
    model_dir.mkdir(parents=True, exist_ok=True)
    
    changing_params = dict(
        model_dir=model_dir,
        band_indices=band_indices,
    )
    params = fixed_params | changing_params
    keras_model_creator.KerasModelCreator(**params).run()


Epoch 1/10


I0000 00:00:1722381096.042474    1423 service.cc:145] XLA service 0x7fa36c004770 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1722381096.042512    1423 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
I0000 00:00:1722381114.515548    1423 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3550/3550 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.4479 - auc: 0.8731 - loss: 0.3630 - macrof1score: 0.4105 - microf1score: 0.6034 - prc: 0.7076 - precision: 0.7271 - recall: 0.5180 - weightedf1score: 0.5657

I0000 00:00:1722381353.694101    1646 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_474', 192 bytes spill stores, 192 bytes spill loads

I0000 00:00:1722381353.733173    1654 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_474', 308 bytes spill stores, 308 bytes spill loads




Epoch 1: val_recall improved from 0.50000 to 0.67263, saving model to models/soilgrid_elevation/binary_crossentropy-237212-7-15-06-64-32-soil/model.keras
3550/3550 ━━━━━━━━━━━━━━━━━━━━ 273s 71ms/step - accuracy: 0.4479 - auc: 0.8731 - loss: 0.3630 - macrof1score: 0.4105 - microf1score: 0.6034 - prc: 0.7076 - precision: 0.7271 - recall: 0.5180 - weightedf1score: 0.5657 - val_accuracy: 0.5324 - val_auc: 0.9248 - val_loss: 0.2914 - val_macrof1score: 0.5649 - val_microf1score: 0.7193 - val_prc: 0.8131 - val_precision: 0.7730 - val_recall: 0.6726 - val_weightedf1score: 0.6954 - learning_rate: 1.0000e-04
Epoch 2/10
3550/3550 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.5155 - auc: 0.9154 - loss: 0.3068 - macrof1score: 0.5186 - microf1score: 0.6913 - prc: 0.7874 - precision: 0.7668 - recall: 0.6295 - weightedf1score: 0.6615
Epoch 2: val_recall did not improve from 0.67263
3550/3550 ━━━━━━━━━━━━━━━━━━━━ 246s 69ms/step - accuracy: 0.5155 - auc: 0.9154 - loss: 0.3068 - macrof1score: 0.5186 -

Evaluate the model for given years and save the results in the model's directory.

In [ ]:
reload(plot_utils)

best_metric = 'val_weightedf1score'
best_rows = []

for name, band_group in zip(names, band_groups):
    model_dirs = list(model_parent_dir.glob(f'*{name}'))
    for model_dir in model_dirs:
        df = pd.read_csv(model_dir.joinpath('model.log'))
        best = df[df[best_metric] == df[best_metric].max()]
        best_rows.append(best)
        
best_df = pd.concat(best_rows).round(2).reset_index(drop=True)
line_cols = ['recall', 'precision', 'weightedf1score']
line_cols += ['prc', 'auc']
fig = px.line(best_df, y=line_cols, x=names)

figure_dir = Path('report', 'figures', 'figures_analysis')
figure_dir.mkdir(exist_ok=True)
fig.write_image(figure_dir.joinpath('soil_elevation_analysis.pdf'))
fig

In [ ]:
# for eval_year in [2017, 2020, 2021, 2022, 2023]:
    # eval_generator = DataGenerator(
        # utils.selected_classes.index, shuffle=False, year=eval_year, **params)
#     preds_path = model_dir.joinpath(f"preds_{params['year']}.npy")
#     if preds_path.is_file():
#         y_pred = np.load(preds_path)
#     else:
#         y_pred = model.predict(x=eval_generator, verbose=1)
#         np.save(preds_path, y_pred)

#     y_true = utils.selected_classes.iloc[:y_pred.shape[0]].to_numpy()
#     cm = confusion_matrix(y_true.flatten(), (y_pred > 0.5).flatten().astype(int))
#     plot = ConfusionMatrixDisplay(confusion_matrix=cm).plot()
#     display(plot)

Visualise the confusion matrix for each class.

In [ ]:
# import matplotlib.pyplot as plt

# class_names = utils.selected_classes.columns

# f, axes = plt.subplots(4, 2, figsize=(25, 30))
# axes = axes.ravel()
# for label in range(y_true.shape[1]):
#     cm = confusion_matrix(y_true[..., label].astype(int), (y_pred[..., label] > 0.5).astype(int))
#     disp = ConfusionMatrixDisplay(cm)
#     disp.plot(ax=axes[label], values_format='.4g')
#     disp.ax_.set_title(f'{class_names[label]}')
#     if label < 25:
#         disp.ax_.set_xlabel('')
#     if label % 5 != 0:
#         disp.ax_.set_ylabel('')
#     disp.im_.colorbar.remove()

# plt.subplots_adjust(wspace=0.2, hspace=0.001)
# f.colorbar(disp.im_, ax=axes)
# plt.show()


In [ ]:
# import tensorflow
# tensorflow.keras.utils.plot_model(model, show_shapes=True)
# model.summary()

Visualise the model.

In [ ]:
# import tensorflow
# tensorflow.keras.utils.plot_model(
#     model, to_file=model_dir.joinpath('model.png'),
#     show_shapes=True, show_layer_activations=True,
# )

Correlate the model with climate variables (to move?)

In [ ]:
# import plotly.express as px

# class_names = utils.selected_classes.columns
# indices = utils.selected_classes.index

# preds_path = model_dir.joinpath(f'preds_2020.npy')
# y_prev = np.load(preds_path)

# weather_prev = pd.read_csv(Path('weather_data', 'era5_2020.csv'))
# eval_years = [2021, 2022, 2023]
# corrs = []

# for eval_year in eval_years:
#     preds_path = model_dir.joinpath(f'preds_{eval_year}.npy')
#     y_pred = np.load(preds_path)
#     y_diff = pd.DataFrame(y_pred - y_prev, columns=class_names)
    
#     weather = pd.read_csv(Path('weather_data', f'era5_{eval_year}.csv'))
#     weather_diff = ((weather - weather_prev)
#                     .loc[indices]
#                     .iloc[:y_pred.shape[0]]
#                     .iloc[y_diff.index])
    
#     corr = y_diff.join(weather_diff).corr(method='pearson').round(2)
#     corrs.append(corr)

#     y_prev = y_pred
#     weather_prev = weather

# fig = px.imshow(
#     np.array(corrs),
#     animation_frame=0,
#     labels=dict(color="Corr coef"),
#     x=corrs[0].index,
#     y=corrs[0].columns,
#     title='Annual correlation heatmap',
#     text_auto=True, aspect='auto', zmin=0, height=500
# )
# fig.layout.sliders[0]['currentvalue']['prefix'] = ''
# for year, step in zip(eval_years, fig.layout.sliders[0].steps):
#     step.label = str(year)

# fig